# This is just a notebook to visualise 1kHz filtered raw data

## Setup everything

### Import packages

In [1]:
import quantities as pq
import numpy as np
import neo
from pathlib import Path
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
import scipy
from scipy import interpolate
from scipy import fftpack
from scipy import signal
%matplotlib widget

from ephyviewer import mkQApp, MainViewer, TraceViewer

Load RawDataChannelExtractedDS.npy

In [15]:
#folder_base = Path('Gaelle/DOR/BlueDots/HabD2/Rec1/OpenEphys/')
folder_base = Path('Gaelle/Baseline_recording/GreenLinesOK/session2/OpenEphys/')


filename = folder_base / f'LFPwake0.npy'
filename2 = folder_base / f'RawDataChannelExtractedDS.npy'

All = np.load(filename, mmap_mode= 'r')
All0 = np.load(filename2, mmap_mode= 'r')

In [16]:
EMG = All[:, 11]
PFC1 = All[:, 22]
PFC2 = All[:, 23]
PFC = PFC2 - PFC1
S11 = All[:, 18]
S12 = All[:, 19]
S1 = S12 - S11
CA11 = All[:, 13]
CA12 = All[:, 15]
CA13 = All[:, 31]
CA1 = CA12 - CA11
#CA1b = CA13 - CA11
#CA1c = CA13 - CA12



Visualise All

In [17]:
start = 000000
end = len(EMG)
EMGs = EMG[start:end, np.newaxis]
PFC1s = PFC1[start:end, np.newaxis]
PFC2s = PFC2[start:end, np.newaxis]
PFCs = PFC[start:end, np.newaxis]
S11s = S11[start:end, np.newaxis]
S12s = S12[start:end, np.newaxis]
S1s = S1[start:end, np.newaxis]
CA11s = CA11[start:end, np.newaxis]
CA12s = CA12[start:end, np.newaxis]
#CA13s = CA13[start:end, np.newaxis]
CA1s = CA1[start:end, np.newaxis]
#CA1s2 = CA1b[start:end, np.newaxis]
#CA1s3 = CA1c[start:end, np.newaxis]
combined = np.append(EMGs, PFC1s, axis=1)
combined = np.append(combined, PFC2s, axis=1)
combined = np.append(combined, PFCs, axis=1)
combined = np.append(combined, S12s, axis=1)
combined = np.append(combined, S11s, axis=1)
combined = np.append(combined, S1s, axis=1)
combined = np.append(combined, CA11s, axis=1)
combined = np.append(combined, CA12s, axis=1)
combined = np.append(combined, CA1s, axis=1)


In [19]:
app = mkQApp()


sample_rate = 1000.
t_start = 0.

#Create the main window that can contain several viewers
win = MainViewer()
view1 = TraceViewer.from_numpy(combined, sample_rate, t_start, 'Signals')
win.add_view(view1)

#Parameters can be set in script
#view1.params['scale_mode'] = 'same_for_all'
view1.params['display_labels'] = True
#And also parameters for each channel
view1.by_channel_params['ch0', 'color'] = '#aa0000'
view1.by_channel_params['ch1', 'color'] = '#0055ff'
view1.by_channel_params['ch2', 'color'] = '#0055ff'
view1.by_channel_params['ch4', 'color'] = '#ff5500'
view1.by_channel_params['ch5', 'color'] = '#ff5500'
view1.by_channel_params['ch7', 'color'] = '#9b3b70'
view1.by_channel_params['ch8', 'color'] = '#9b3b70'
#view1.by_channel_params['ch9', 'color'] = '#9b3b70'



view1.by_channel_params['ch0', 'gain'] = 0.00002
view1.by_channel_params['ch1', 'gain'] = 0.00002
view1.by_channel_params['ch2', 'gain'] = 0.00002
view1.by_channel_params['ch3', 'gain'] = 0.00006
view1.by_channel_params['ch4', 'gain'] = 0.00002
view1.by_channel_params['ch5', 'gain'] = 0.00002
view1.by_channel_params['ch6', 'gain'] = 0.00003
view1.by_channel_params['ch7', 'gain'] = 0.00002
view1.by_channel_params['ch8', 'gain'] = 0.00002
view1.by_channel_params['ch9', 'gain'] = 0.00002

view1.by_channel_params['ch0', 'offset'] = 0.5
view1.by_channel_params['ch1', 'offset'] = 0.4
view1.by_channel_params['ch2', 'offset'] = 0.3
view1.by_channel_params['ch3', 'offset'] = 0.2
view1.by_channel_params['ch4', 'offset'] = 0.1
view1.by_channel_params['ch5', 'offset'] = 0
view1.by_channel_params['ch6', 'offset'] = -0.1
view1.by_channel_params['ch7', 'offset'] = -0.2
view1.by_channel_params['ch8', 'offset'] = -0.3
view1.by_channel_params['ch9', 'offset'] = -0.4

#Run
win.show()
app.exec_()

0

In [132]:
print(len(EMG))

2733405


: 

Visualise subset

In [10]:

start = 000000
end = len(EMG)
combined = np.append(EMGs, CA11s, axis=1)
#combined = np.append(combined, PFC2s, axis=1)
#combined = np.append(combined, PFCs, axis=1)
#combined = np.append(combined, S12s, axis=1)
#combined = np.append(combined, S11s, axis=1)
#combined = np.append(combined, S1s, axis=1)
#combined = np.append(combined, S1s, axis=1)
#combined = np.append(combined, CA11s, axis=1)
combined = np.append(combined, CA12s, axis=1)
combined = np.append(combined, CA13s, axis=1)
combined = np.append(combined, CA1s, axis=1) #CA12 - CA11
combined = np.append(combined, CA1s2, axis=1) #CA13 - CA11
combined = np.append(combined, CA1s3, axis=1) #CA13 - CA12
app = mkQApp()


sample_rate = 1000.
t_start = 0.

#Create the main window that can contain several viewers
win = MainViewer()
view1 = TraceViewer.from_numpy(combined, sample_rate, t_start, 'Signals')
win.add_view(view1)

#Parameters can be set in script
#view1.params['scale_mode'] = 'same_for_all'
view1.params['display_labels'] = True
#And also parameters for each channel
view1.by_channel_params['ch0', 'color'] = '#aa0000'
view1.by_channel_params['ch1', 'color'] = '#9b3b70'
#view1.by_channel_params['ch2', 'color'] = '#0055ff'
#view1.by_channel_params['ch5', 'color'] = '#ff5500'
view1.by_channel_params['ch2', 'color'] = '#9b3b70'
#view1.by_channel_params['ch3', 'color'] = '#9b3b70'
view1.by_channel_params['ch3', 'color'] = '#9b3b70'
#view1.by_channel_params['ch4', 'color'] = '#9b3b70'



view1.by_channel_params['ch0', 'gain'] = 0.00002
view1.by_channel_params['ch1', 'gain'] = 0.00001
view1.by_channel_params['ch2', 'gain'] = 0.00001
view1.by_channel_params['ch3', 'gain'] = 0.00001
view1.by_channel_params['ch4', 'gain'] = 0.00002
view1.by_channel_params['ch5', 'gain'] = 0.00002
#view1.by_channel_params['ch5', 'gain'] = 0.00003
view1.by_channel_params['ch6', 'gain'] = 0.00002
# view1.by_channel_params['ch7', 'gain'] = 0.00001
# view1.by_channel_params['ch8', 'gain'] = 0.00001
# view1.by_channel_params['ch9', 'gain'] = 0.00005

view1.by_channel_params['ch0', 'offset'] = 0.3
view1.by_channel_params['ch1', 'offset'] = 0.2
view1.by_channel_params['ch2', 'offset'] = 0.1
view1.by_channel_params['ch3', 'offset'] = -0.
view1.by_channel_params['ch4', 'offset'] = -0.1
view1.by_channel_params['ch5', 'offset'] = -0.2
#view1.by_channel_params['ch5', 'offset'] = -0.2
view1.by_channel_params['ch6', 'offset'] = -0.3
# view1.by_channel_params['ch7', 'offset'] = -0.2
# view1.by_channel_params['ch8', 'offset'] = -0.3
# view1.by_channel_params['ch9', 'offset'] = -0.4

#Run
win.show()
app.exec_()

NameError: name 'CA13s' is not defined

In [25]:
folder_base = Path('Gaelle/Baseline_recording/GreenLinesOK/session3/OpenEphys/')


filename = folder_base / f'LFPwake0.npy'
filename2 = folder_base / f'RawDataChannelExtractedDS.npy'

All = np.load(filename2, mmap_mode= 'r')
All0 = np.load(filename, mmap_mode= 'r')

combined0 = All[:,24:32].copy()
combined1 = All0[:,0:8].copy()
combined2 = np.append(combined0, combined1, axis=1)
combined2.shape
# Rec2 (916030, 16)

(910224, 16)

In [26]:
app = mkQApp()


sample_rate = 1000.
t_start = 0.

#Create the main window that can contain several viewers
win = MainViewer()
view1 = TraceViewer.from_numpy(combined2, sample_rate, t_start, 'Signals')
win.add_view(view1)

#Parameters can be set in script
#view1.params['scale_mode'] = 'same_for_all'
view1.params['display_labels'] = True
#And also parameters for each channel

view1.by_channel_params['ch0', 'gain'] = 0.00002
view1.by_channel_params['ch1', 'gain'] = 0.00002
view1.by_channel_params['ch2', 'gain'] = 0.00002
view1.by_channel_params['ch3', 'gain'] = 0.00002
view1.by_channel_params['ch4', 'gain'] = 0.00002
view1.by_channel_params['ch5', 'gain'] = 0.00002
view1.by_channel_params['ch6', 'gain'] = 0.00002
view1.by_channel_params['ch7', 'gain'] = 0.00002
view1.by_channel_params['ch8', 'gain'] = 0.00002
view1.by_channel_params['ch9', 'gain'] = 0.00002
view1.by_channel_params['ch10', 'gain'] = 0.00002
view1.by_channel_params['ch11', 'gain'] = 0.00002
view1.by_channel_params['ch12', 'gain'] = 0.00002
view1.by_channel_params['ch13', 'gain'] = 0.00002
view1.by_channel_params['ch14', 'gain'] = 0.00002
view1.by_channel_params['ch15', 'gain'] = 0.00002

view1.by_channel_params['ch0', 'offset'] = 0.8
view1.by_channel_params['ch1', 'offset'] = 0.7
view1.by_channel_params['ch2', 'offset'] = 0.6
view1.by_channel_params['ch3', 'offset'] = 0.5
view1.by_channel_params['ch4', 'offset'] = 0.4
view1.by_channel_params['ch5', 'offset'] = 0.3
view1.by_channel_params['ch6', 'offset'] = 0.2
view1.by_channel_params['ch7', 'offset'] = 0.1
view1.by_channel_params['ch8', 'offset'] = 0.
view1.by_channel_params['ch9', 'offset'] = -0.1
view1.by_channel_params['ch10', 'offset'] = -0.2
view1.by_channel_params['ch11', 'offset'] = -0.3
view1.by_channel_params['ch12', 'offset'] = -0.4
view1.by_channel_params['ch13', 'offset'] = -0.5
view1.by_channel_params['ch14', 'offset'] = -0.6
view1.by_channel_params['ch15', 'offset'] = -0.7

view1.by_channel_params['ch0', 'color'] = '#0055ff'
view1.by_channel_params['ch1', 'color'] = '#0055ff'
view1.by_channel_params['ch2', 'color'] = '#0055ff'
view1.by_channel_params['ch3', 'color'] = '#0055ff'
view1.by_channel_params['ch4', 'color'] = '#0055ff'
view1.by_channel_params['ch5', 'color'] = '#0055ff'
view1.by_channel_params['ch6', 'color'] = '#0055ff'
view1.by_channel_params['ch7', 'color'] = '#0055ff'

#Run
win.show()
app.exec_()


0